In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, cross_val_score

In [2]:
try:
    train_df = pd.read_csv("train.csv")
    test_df = pd.read_csv("test.csv")
    print("Arquivos 'train.csv' e 'test.csv' carregados com sucesso!")
except FileNotFoundError:
    print("\nERRO: Arquivo não encontrado!")

Arquivos 'train.csv' e 'test.csv' carregados com sucesso!


In [3]:
print("Preparando dados com a abordagem simples.")
features_to_drop = ['labels', 'id', 'category_code']
X = train_df.drop(columns=features_to_drop)
y = train_df['labels']

Preparando dados com a abordagem simples.


In [4]:
print("\n--- Etapa de Análise de Importância ---")
# Criamos um pipeline temporário apenas para treinar o modelo e ver as importâncias
temp_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(n_estimators=200, random_state=42, class_weight='balanced', n_jobs=-1))
])
temp_pipeline.fit(X, y)

# Extrair as importâncias do modelo treinado
importances = temp_pipeline.named_steps['classifier'].feature_importances_
feature_names = X.columns

# Criar um DataFrame para visualizar as importâncias
feature_importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

print("Ranking de Features por Importância:")
print(feature_importance_df)


--- Etapa de Análise de Importância ---
Ranking de Features por Importância:
                     feature  importance
4              relationships    0.140269
3    age_last_milestone_year    0.116542
6          funding_total_usd    0.108256
1      age_last_funding_year    0.093269
0     age_first_funding_year    0.085942
2   age_first_milestone_year    0.082207
29          avg_participants    0.074409
7                 milestones    0.065790
5             funding_rounds    0.041352
26                has_roundB    0.016033
12             is_otherstate    0.014719
8                      is_CA    0.014168
23                    has_VC    0.014160
22          is_othercategory    0.013709
27                has_roundC    0.013650
25                has_roundA    0.012461
13               is_software    0.010837
10                     is_MA    0.010339
28                has_roundD    0.009816
14                    is_web    0.008731
24                 has_angel    0.008394
16             is_en

In [5]:
print("\n--- Etapa de Validação com Features Selecionadas ---")

# Vamos selecionar as 15 features mais importantes
N_TOP_FEATURES = 15
top_features = feature_importance_df['feature'].head(N_TOP_FEATURES).tolist()

print(f"\nTestando o modelo usando apenas as {N_TOP_FEATURES} features mais importantes...")
X_selected = X[top_features]

# Usar o mesmo pipeline e a mesma validação cruzada de antes
cv = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(temp_pipeline, X_selected, y, cv=cv, scoring='accuracy')

print("\n--- RESULTADO DA VALIDAÇÃO (COM FEATURES SELECIONADAS) ---")
print(f"Acurácias de cada um dos 5 folds: {np.round(scores, 4)}")
print(f"Acurácia Média: {scores.mean():.4f}")
print("----------------------------------------------------------")
print("Compare este resultado com a nossa melhor média anterior (~0.7787).")


--- Etapa de Validação com Features Selecionadas ---

Testando o modelo usando apenas as 15 features mais importantes...

--- RESULTADO DA VALIDAÇÃO (COM FEATURES SELECIONADAS) ---
Acurácias de cada um dos 5 folds: [0.7462 0.7442 0.7597 0.8372 0.8295]
Acurácia Média: 0.7833
----------------------------------------------------------
Compare este resultado com a nossa melhor média anterior (~0.7787).


In [6]:
print("\n--- Etapa de Submissão ---")
print("Treinando o modelo final com as features selecionadas...")
temp_pipeline.fit(X_selected, y) # Treina com os dados e features selecionados

# Prepara o test set com as mesmas features
test_ids = test_df['id']
X_test_selected = test_df[top_features] # Seleciona as mesmas colunas no teste
X_selected, X_test_selected = X_selected.align(X_test_selected, join='left', axis=1, fill_value=0)

final_predictions = temp_pipeline.predict(X_test_selected)
submission_df = pd.DataFrame({'id': test_ids, 'labels': final_predictions})
submission_df.to_csv('final_submission_feature_selection.csv', index=False)

print("\nArquivo 'final_submission_feature_selection.csv' criado com sucesso!")


--- Etapa de Submissão ---
Treinando o modelo final com as features selecionadas...

Arquivo 'final_submission_feature_selection.csv' criado com sucesso!
